# Building inverted index and answering queries

In the first part of the lab you are going to implement a standard document processing pipeline and then build a simple search engine based on it:
- starting from crawling documents,
- then building an inverted index,
- and answering queries using this index.

## Preprocessing

First, we need a unified approach to documents and queries preprocessing. Implement a class responsible for that. Complete the code for given functions (most of them are just one-liners) and make sure you pass the tests. Make use of `nltk` library, `spacy`, or any other you know.

In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
import nltk

class Preprocessor:

    def __init__(self):
        self.stop_words = {'a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'for', 'from', 'has', 'he', 'in', 'is', 'it', 'its',
                      'of', 'on', 'that', 'the', 'to', 'was', 'were', 'will', 'with'}
        self.ps = nltk.stem.PorterStemmer()


    def tokenize(self, text):
        #TODO word tokenize text using nltk lib

        return ['one', 'two', 'three']


    def stem(self, word, stemmer):

        #TODO stem word using provided stemmer

        return 'stemmed_word'


    def is_apt_word(self, word):
        #TODO check if word is appropriate - not a stop word and isalpha,
        # i.e consists of letters, not punctuation, numbers, dates

        return False


    def preprocess(self, text):
        # and stem it, ignoring not appropriate words


        return ['one', 'two', 'three']

### Tests

In [5]:
prep = Preprocessor()
text = 'To be, or not to be, that is the question'

assert prep.tokenize(text) == ['To', 'be', ',', 'or', 'not', 'to', 'be', ',', 'that', 'is', 'the', 'question']
assert prep.stem('retrieval', prep.ps) == 'retriev'
assert prep.is_apt_word('qwerty123') is False
assert prep.preprocess(text) == ['or', 'not', 'question']

## Crawling and Indexing

### Base classes

Here are some base classes you will need for writing an indexer. The code is given, still, you need to change some. Namely, the `parse` method (it is also possible to use your own implementations of other methods, just make sure they work). The reason to change is that the method always makes complete parsing, which we want to avoid, when we only need e.g. links or a specific portions of text.

In [16]:
import requests
from urllib.parse import quote
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.parse
import os


class Document:

    def __init__(self, url):
        self.url = url

    def download(self):
        try:
            response = requests.get(self.url)
            if response.status_code == 200:
                self.content = response.content
                return True
            else:
                return False
        except:
            return False

    def persist(self, path):
        # this code is not supposed to be good :)
        # Please discuss why this line is bad
        with open(os.path.join(path, quote(self.url).replace('/', '_')), 'wb') as f:
            f.write(self.content)


class HtmlnbcnewsArticle(Document):

    def normalize(self, href):
        if href is not None and href[:4] != 'http':
            href = urllib.parse.urljoin(self.url, href)
        return href

    def parse(self):
        #TODO change this method

        def tag_visible(element):
            if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
                return False
            if isinstance(element, Comment):
                return False
            return True

        model = BeautifulSoup(self.content, "html.parser")

        self.anchors = []
        a = model.find_all('a')
        for anchor in a:
            href = self.normalize(anchor.get('href'))
            text = anchor.text
            self.anchors.append((text, href))

        # extract only header and main content
        # discuss why using classes like article-body__content__17Yit
        # is the wrong strategy
        header = ### TODO
        content = ### TODO


        if header is None or content is None:
          self.content = content
        if content:
            content = content.parent
        else:

            content = model.find('p')
            print(f'article body not found: {content}')

        if header is None or content is None:
            self.article_text = ""
            return

        texts = header.findAll(string=True) + content.findAll(string=True)
        visible_texts = filter(tag_visible, texts)
        self.article_text = "\n".join(t.strip() for t in visible_texts)

In [17]:
doc = HtmlnbcnewsArticle("https://www.nbcnews.com/news/us-news/jeff-bezos-sells-nearly-12-million-amazon-shares-least-2-billion-come-rcna138274")
doc.download()

doc.parse()
print(doc.article_text)

Jeff Bezos sells nearly 12 million Amazon shares worth at least $2 billion, with more to come


### Main class

The main indexer logic is here. We organize it as a crawler generator that adds certain visited pages to inverted index and saves them on disk.

- `crawl_generator_for_index` method crawles the given website doing BFS, starting from `source` within given `depth`. Considers only inner pages (starting with https://www.nbcnews.com/...) for visiting. To speed up, do not consider pages with content type other than `html`: `.pdf`, `.mp3`, `.avi`, `.mp4`, `.txt`, ... . If crawler encounters an article page (of a form https://www.nbcnews.com/world/...), it saves its content in a file in `collection_path` folder and populates the inverted index calling `index_doc` method. When done, saves on disk three resulting dictionaries:
    - `doc_urls`: `{doc_id : url}`
    - `index`: `{term : [collection_frequency, (doc_id_1, doc_freq_1), (doc_id_2, doc_freq_2), ...]}`
    - `doc_lengths`: `{doc_id : doc_length}`

    `limit` parameter is given for testing - if not `None`, break the loop when number of saved articles exceeds the `limit` and return without writing dictionaries to disk.
    
- `index_doc` method parses and preprocesses the content of a `doc` and adds it to the inverted index. Also keeps track of document lengths in a `doc_lengths` dictionary.

Your crawler have to print visited urls as it runs.

In [19]:
from collections import Counter
from queue import Queue
import pickle
import os
import re

class nbcnewsSpecificIndexer:

    def __init__(self):
        # dictionaries to populate
        self.doc_urls = {}
        self.index = {}
        self.doc_lengths = {}
        # preprocessor
        self.prep = Preprocessor()


    def crawl_generator_for_index(self, source, depth, collection_path="collection", limit=None):

        q = Queue()
        q.put((source, 0))
        visited = set()
        doc_counter = 0
        # creating a folder if needed
        if not os.path.exists(collection_path):
            os.makedirs(collection_path)
        # doing a BFS
        while not q.empty():
            url, url_depth = q.get()
            if url not in visited:
                visited.add(url)
                try:
                    doc = HtmlnbcnewsArticle(url)    # download and parse url
                    if doc.download():
                        doc.parse()
                        ### TODO write a regular expression that will match only webpages under nbcnews domain
                        if re.match(r'...', url):

                        # if url.startswith("https://www.nbcnews.com/"):
                            # print(url)
                            doc.persist(collection_path)
                            self.doc_urls[doc_counter] = url
                            self.index_doc(doc, doc_counter)
                            doc_counter += 1
                            yield doc
                            if limit is not None and doc_counter == limit:
                                return

                            # filter links, consider only inner html pages
                        if url_depth + 1 < depth:
                            valid_anchors = filter(self.accepted_url, doc.anchors)
                            for a in valid_anchors:
                                q.put((a[1], url_depth + 1))

                except FileNotFoundError as e:
                    print("Analyzing", url, "led to FileNotFoundError")


    def accepted_url(self, anchor):
        url = str(anchor[1])
        if not url.startswith("https://www.nbcnews.com"):
            return False
        if url[-4:]  in ('.pdf', '.mp3', '.avi', '.mp4', '.txt'):
            return False
        return True


    def index_doc(self, doc, doc_id):
        # add documents to index
        doc.parse()
        # preprocess - tokenize, remove stopwords and non-alphanumeric tokens and stem
        content = self.prep.preprocess(doc.article_text)
        self.doc_lengths[doc_id] = len(content)
        # get dict of terms in current article
        article_index = Counter(content)
        # update global index
        for term in article_index.keys():
            article_freq = article_index[term]
            if term not in self.index:
                self.index[term] = [article_freq, (doc_id, article_freq)]
            else:
                self.index[term][0] += article_freq
                self.index[term].append((doc_id, article_freq))

### Tests

Please make sure your crawler prints out urls with `print(k, c.url)`

In [20]:
import requests

requests.get('https://www.nbcnews.com/news/us-news/jeff-bezos-sells-nearly-12-million-amazon-shares-least-2-billion-come-rcna138274').status_code

200

In [21]:
indexer = nbcnewsSpecificIndexer()
k = 1

for c in indexer.crawl_generator_for_index(
        source="https://www.nbcnews.com/news",
        depth=2,
        collection_path="test_collection",
        limit=15):
    print(k, c.url)
    k += 1


# assert type(indexer.index) is dict
# assert type(indexer.index['nbcnews']) is list
# assert type(indexer.index['nbcnews'][0]) is int
# assert type(indexer.index['nbcnews'][1]) is tuple

1 https://www.nbcnews.com/politics/2024-election/live-blog/harris-trump-presidential-election-live-updates-rcna171168
2 https://www.nbcnews.com/news/us-news/nbc-affiliates-n19981
3 https://www.nbcnews.com/news/weather
4 https://www.nbcnews.com/politics/2024-presidential-election
5 https://www.nbcnews.com/news/nbc-news-now-live-audio-listen-live-news-audio-day-rcna70163
6 https://www.nbcnews.com/news/us-news/sean-diddy-combs-arrested-rcna145503
7 https://www.nbcnews.com/news/us-news/sean-diddy-combs-files-motion-vacate-100m-default-judgment-sexual-assa-rcna170430
8 https://www.nbcnews.com/news/us-news/former-danity-kane-member-sues-sean-combs-alleging-groped-threatened-rcna170617
9 https://www.nbcnews.com/news/us-news/diddy-lawsuits-timeline-allegations-what-know-rcna145335
10 https://www.nbcnews.com/investigations/secret-service-chief-makes-remarkable-admission-need-paradigm-shift-rcna171390
11 https://www.nbcnews.com/politics/donald-trump/congress-mulls-new-secret-service-funding-appa

Please test these documents contain a desired stem (or its derivate):

In [22]:
indexer.index.keys()

dict_keys(['elect', 'updat', 'trump', 'make', 'virtual', 'appear', 'crypto', 'event', 'harri', 'meet', 'teamster', 'nbc', 'affili', 'weather', 'news', 'now', 'live', 'audio', 'sean', 'comb', 'arrest', 'feder', 'agent', 'new', 'york', 'file', 'motion', 'vacat', 'default', 'judgment', 'sexual', 'assault', 'lawsuit', 'inmat', 'former', 'daniti', 'kane', 'member', 'sue', 'alleg', 'grope', 'threaten', 'her', 'timelin', 'what', 'know', 'secret', 'servic', 'chief', 'remark', 'admiss', 'we', 'need', 'paradigm', 'shift', 'congress', 'mull', 'fund', 'after', 'appar', 'attempt', 'life', 'about', 'ryan', 'routh', 'man', 'charg', 'assassin', 'golf', 'cours', 'suspect', 'second', 'plot', 'camp', 'out', 'hour', 'did', 'have', 'hi', 'line', 'sight', 'when', 'fire', 'him', 'video', 'show', 'be'])

In [23]:
some_stem = 'live'
print(indexer.index[some_stem])
for pair in indexer.index[some_stem][1:]:
    print(indexer.doc_urls[pair[0]])

[1, (4, 1)]
https://www.nbcnews.com/news/nbc-news-now-live-audio-listen-live-news-audio-day-rcna70163


### 1.2.4. Building an index

In [24]:
indexer = nbcnewsSpecificIndexer()
for k, c in enumerate(
                indexer
                    .crawl_generator_for_index(
                        "https://www.nbcnews.com/",
                        3,
                        "docs_collection",
                        limit=100   # optional limit
                    )):
    print(k + 1, c.url)

1 https://www.nbcnews.com/politics/2024-election/live-blog/harris-trump-presidential-election-live-updates-rcna171168
2 https://www.nbcnews.com/news/us-news/nbc-affiliates-n19981
3 https://www.nbcnews.com/news/weather
4 https://www.nbcnews.com/politics/2024-presidential-election
5 https://www.nbcnews.com/news/nbc-news-now-live-audio-listen-live-news-audio-day-rcna70163
6 https://www.nbcnews.com/news/us-news/sean-diddy-combs-arrested-rcna145503
7 https://www.nbcnews.com/news/us-news/sean-diddy-combs-files-motion-vacate-100m-default-judgment-sexual-assa-rcna170430
8 https://www.nbcnews.com/news/us-news/former-danity-kane-member-sues-sean-combs-alleging-groped-threatened-rcna170617
9 https://www.nbcnews.com/news/us-news/diddy-lawsuits-timeline-allegations-what-know-rcna145335
10 https://www.nbcnews.com/investigations/secret-service-chief-makes-remarkable-admission-need-paradigm-shift-rcna171390
11 https://www.nbcnews.com/politics/donald-trump/congress-mulls-new-secret-service-funding-appa

### Index statistics

Load the index and print the statistics.

In [31]:
print('Total index keys count', len(indexer.index))

print('\nTop stems by number of documents they apperared in:')
sorted_by_n_docs = sorted(indexer.index.items(), key=lambda kv: (len(kv[1]), kv[0]), reverse=True)
print([(sorted_by_n_docs[i][0], len(sorted_by_n_docs[i][1])-1) for i in range(20)])

print('\nTop stems by overall frequency:')
sorted_by_freq = sorted(indexer.index.items(), key=lambda kv: (kv[1][0], kv[0]), reverse=True)
print([(sorted_by_freq[i][0], sorted_by_freq[i][1][0]) for i in range(20)])

Total index keys count 549

Top stems by number of documents they apperared in:
[('care', 18), ('skin', 17), ('after', 16), ('trump', 15), ('know', 7), ('harri', 7), ('nbc', 6), ('elect', 6), ('attempt', 6), ('what', 5), ('week', 5), ('say', 5), ('new', 5), ('make', 5), ('crypto', 5), ('best', 5), ('appar', 5), ('american', 5), ('about', 5), ('virtual', 4)]

Top stems by overall frequency:
[('care', 18), ('skin', 17), ('after', 16), ('trump', 15), ('know', 7), ('harri', 7), ('nbc', 6), ('elect', 6), ('attempt', 6), ('what', 5), ('week', 5), ('say', 5), ('s', 5), ('new', 5), ('make', 5), ('first', 5), ('crypto', 5), ('best', 5), ('appar', 5), ('american', 5)]


## Answering a query (finally)

Now, given that we already have built the inverted index, it's time to utilize it for answering user queries. In this class there are two methods you need to implement:
- `boolean_retrieval`, the simplest form of document retrieval which returns a set of documents such that each one contains all query terms. Returns a set of document ids. Refer to *ch.1* of the book for details;
- `okapi_scoring`, Okapi BM25 ranking function - assigns scores to documents in the collection that are relevant to the user query. Returns a dictionary of scores, `doc_id:score`. Read about it in [Wikipedia](https://en.wikipedia.org/wiki/Okapi_BM25#The_ranking_function) and implement accordingly.

Both methods accept `query` parameter in a form of a dictionary, `term:frequency`

In [26]:
from collections import Counter
import math

class QueryProcessing:

    @staticmethod
    def prepare_query(raw_query):
        prep = Preprocessor()
        # pre-process query the same way as documents
        query = prep.preprocess(raw_query)
        # count frequency
        return Counter(query)

    @staticmethod
    def boolean_retrieval(query, index):


        # retrieve a set of documents containing all query terms
        #TODO retrieve a set of documents containing all query terms
        RetDocs = []
        .
        .
        .
        docs = set.intersection(*map(set,RetDocs))
        return docs

        return {0, 1, 3}


    @staticmethod
    def okapi_scoring(query, doc_lengths, index, k1=1.2, b=0.75):
        #TODO retrieve relevant documents with scores

        scores = {}
        N = len(doc_lengths)
        avgdl = sum(doc_lengths.values()) / float(len(doc_lengths))
        for term in query.keys():
            if term not in index:  # ignoring absent terms
                continue
            n_docs = ### TODO
            idf = ### TODO
            postings = index[term][1:]
            for posting in postings:
                doc_id = ### TODO
                doc_tf = ### TODO
                score = ### TODO
                if doc_id not in scores:
                    scores[doc_id] = score
                else:  # accumulate scores
                    scores[doc_id] += score
        return scores
        return {0: 0.32, 5: 1.17}

### Tests

In [27]:
test_doc_lengths = {1: 20, 2: 15, 3: 10, 4:20, 5:30}
test_index = {'x': [2, (1, 1), (2, 1)], 'y': [2, (1, 1), (3, 1)], 'z': [3, (2, 1), (4,2)]}


test_query1 = QueryProcessing.prepare_query('x z')
test_query2 = QueryProcessing.prepare_query('x y')


assert QueryProcessing.boolean_retrieval(test_query1, test_index) == {2}
assert QueryProcessing.boolean_retrieval(test_query2, test_index) == {1}
okapi_res = QueryProcessing.okapi_scoring(test_query2, test_doc_lengths, test_index)
assert all(k in okapi_res for k in (1, 2, 3))
assert not any(k in okapi_res for k in (4, 5))
assert okapi_res[1] > okapi_res[3] > okapi_res[2]

### And now query the real index

In [29]:
import time

# queries = ["russia missle" , "isreal palestine", "taylor swift"]
queries = ['skin care', 'president elecetion']
for q in queries:
    print(q)
    qobj = QueryProcessing.prepare_query(q)
    for res in QueryProcessing.boolean_retrieval(qobj, indexer.index):
        print('\t', indexer.doc_urls[res])

    s = time.time()
    okapi_res = QueryProcessing.okapi_scoring(qobj, indexer.doc_lengths, indexer.index)
    e = time.time()
    print(f"\t == Okapi Time: {e - s:.5f} ==")
    for res in okapi_res:
        print('\t', indexer.doc_urls[res], okapi_res[res])

    print()

skin care
	 https://www.nbcnews.com/select/shopping/target-circle-week-announcement-oct-2024-rcna171322
	 https://www.nbcnews.com/select/deals-sales
	 https://www.nbcnews.com/select/shopping/100-best-face-moisturizers-rcna151958
	 https://www.nbcnews.com/select/shopping/weekly-sales-rcna123623
	 https://www.nbcnews.com/select/deals-sales/weekly-sales
	 https://www.nbcnews.com/select/shopping/sams-club-fall-membership-deal-rcna167230
	 https://www.nbcnews.com/select/Paid
	 https://www.nbcnews.com/select/shopping/best-board-games-kids-ncna1266531
	 https://www.nbcnews.com/select/home-kitchen
	 https://www.nbcnews.com/select/shopping/amazon-prime-benefits-cost-ncna1269672
	 https://www.nbcnews.com/select/prime-day
	 https://www.nbcnews.com/select/shopping/best-printers-rcna171286
	 https://www.nbcnews.com/select/tech/electronics
	 https://www.nbcnews.com/select/wellness/sleep
	 https://www.nbcnews.com/select/shopping/how-to-shop-for-a-mattress-online-rcna171284
	 https://www.nbcnews.com/s